# Radiative electric field

Given by the equation
$$
\pmb{E}_\text{rad}=\frac{1}{4\pi\epsilon_0}\left(\frac{q\pmb{\hat{R}}\times[(\pmb{\hat{R}}-\pmb{\beta})\times\dot{\pmb{\beta}}]}{c(1-\pmb{\hat{R}}\cdot\pmb{\beta})^3R}\right)_{\text{retarded}},
$$
where $\pmb{R}$ is the vector from the source to the field point
$$
\pmb{R} = \pmb{r} - \pmb{r}^\prime,
$$
and where the time-dependent terms are evaluated at retarded time
$$
t^\prime = t - \frac{R}{c}.
$$

## Definitions
For this specific case, the source point vector $\pmb{r}^\prime$ is given by
$$
\pmb{r}^\prime = r(\pmb{\hat{x}}\cos{\phi} + \pmb{\hat{y}}\sin{\phi}),
$$
where $\phi$ is the angular position of the particle. The relative velocity $\pmb{\beta}$ is perpendicular to the sorce point vector and is given by
$$
\pmb{\beta} = \frac{v}{c}(\pmb{\hat{y}}\cos{\phi}-\pmb{\hat{x}}\sin{\phi}).
$$
The relative acceleration is defined in terms of relative speed $\beta$ and magnetic field magnitude $B$ as
$$
\pmb{\dot{\beta}} = -\frac{qB\beta}{m}(\pmb{\hat{x}}\cos{\phi} + \pmb{\hat{y}}\sin{\phi}).
$$
The magnitude of the magnetic field is variable, dependent on speed and radius, thus the relative acceleration can be redefined in terms of these variables,
$$
\pmb{\dot{\beta}} = -\frac{v^2}{cr}(\pmb{\hat{x}}\cos{\phi} + \pmb{\hat{y}}\sin{\phi}).
$$

## Implementation
The radiative electric field is divided into its components
$$
\pmb{E}_\text{rad} = u(x,y,t)\pmb{\hat{x}}+v(x,y,t)\pmb{\hat{y}}.
$$
The *coupling* constant is merged into a single number,
$$
k = \frac{1}{4\pi\epsilon_0},
$$
and the retarded term of the equation is evaluated, starting with the numerator. For the sake of simplicity, the unit separation vector $\pmb{\hat{R}}$ is given its own symbol,
$$
\pmb{n} \equiv \pmb{\hat{R}}.
$$
The difference between the relative velocity and the unit separation vector expands to
$$
\pmb{n}-\pmb{\beta} = (n_x - \beta_x)\pmb{\hat{x}} + (n_y - \beta_y)\pmb{\hat{y}}.
$$
The first cross product is expanded,
$$
(\pmb{n}-\pmb{\beta})\times\dot{\pmb{\beta}}=\begin{vmatrix}
\pmb{\hat{x}} & \pmb{\hat{y}} & \pmb{\hat{z}} \\ 
(n_x - \beta_x) & (n_y - \beta_y) & 0 \\ 
\dot{\beta}_x & \dot{\beta}_y & 0
\end{vmatrix} = [(n_x - \beta_x)\dot{\beta}_y-(n_y - \beta_y)\dot{\beta}_x]\pmb{\hat{z}}.
$$
To expand the second cross product, the scalar coefficient is brought before the cross product, leaving the cross product between the unit separation vector and the $z$ unit vector,
$$
\pmb{n}\times[(\pmb{n}-\pmb{\beta})\times\dot{\pmb{\beta}}]= [(n_x - \beta_x)\dot{\beta}_y-(n_y - \beta_y)\dot{\beta}_x](\pmb{n}\times\pmb{\hat{z}}).
$$
The resulting cross product is expanded,
$$
\pmb{n}\times\pmb{\hat{z}}=\begin{vmatrix}
\pmb{\hat{x}} & \pmb{\hat{y}} & \pmb{\hat{z}} \\ 
n_x & n_y & 0 \\ 
0 & 0 & 1
\end{vmatrix} = n_y\pmb{\hat{x}} + n_x\pmb{\hat{y}},
$$
and is multiplied by the scalar coefficient and the charge of the particle, giving
$$
\pmb{\text{numerator}} = q[(n_x - \beta_x)\dot{\beta}_y-(n_y - \beta_y)\dot{\beta}_x](n_y\pmb{\hat{x}} + n_x\pmb{\hat{y}}).
$$
In the denominator, only the dot product must be expanded, giving
$$
\text{denominator} = c(1 - n_x\beta_x - n_y\beta_y)^3R.
$$
The components $u(x,y,t)$ and $v(x,y,t)$ are then given by
$$
u(x,y,t) = k\left(\frac{q[(n_x - \beta_x)\dot{\beta}_y-(n_y - \beta_y)\dot{\beta}_x]n_y}{c(1 - n_x\beta_x - n_y\beta_y)^3R}\right)_\text{retarded}
$$
and
$$
v(x,y,t) = k\left(\frac{q[(n_x - \beta_x)\dot{\beta}_y-(n_y - \beta_y)\dot{\beta}_x]n_x}{c(1 - n_x\beta_x - n_y\beta_y)^3R}\right)_\text{retarded}
$$
respectively.

## Vector field

In [1]:
import numpy as np
from ipywidgets import interact
from vectorfields import CustomUV2D

c = 299792458
k = 1/(4 * np.pi * 8.85418782e-12)
q = 2 * 1.60217662e-19
r = 0.5
v = 20e6
frq = v/r

beta = v/c
accl = v**2/(r * c)

def R(x,y,pos):
    return np.sqrt(
        (x - r*np.cos(pos))**2 + (y - r*np.sin(pos))**2
    )

def u(x,y,t):
    _pos = frq * t
    
    pos = frq * (t - R(x,y,_pos)/c)
    
    RR = R(x,y,pos)
    
    nx = (x - r*np.cos(pos))/RR
    ny = (y - r*np.sin(pos))/RR
    
    vx = -beta * np.sin(pos)
    vy =  beta * np.cos(pos)
    
    ax = -accl * np.cos(pos)
    ay = -accl * np.sin(pos)
    
    num = k * q * ( (nx - vx)*ay - (ny - vy)*ax ) * ny
    den = c * ( 1 - nx*vx - ny*vy )**3 * RR
    
    return num/den

def v(x,y,t):
    _pos = frq * t
    
    pos = frq * (t - R(x,y,_pos)/c)
    
    RR = R(x,y,pos)
    
    nx = (x - r*np.cos(pos))/RR
    ny = (y - r*np.sin(pos))/RR
    
    vx = -beta * np.sin(pos)
    vy =  beta * np.cos(pos)
    
    ax = -accl * np.cos(pos)
    ay = -accl * np.sin(pos)
    
    num = k * q * ( (nx - vx)*ay - (ny - vy)*ax ) * nx
    den = c * ( 1 - nx*vx - ny*vy )**3 * RR
    
    return num/den
    
    
def field(t):
    t = t / 10000000
    ufunc = lambda x,y: u(x,y,t)*np.sqrt(x**2+y**2)
    vfunc = lambda x,y: v(x,y,t)*np.sqrt(x**2+y**2)
    
    vf = CustomUV2D(ufunc, vfunc, size=100, resolution=100)
    return vf.plot()

interact(field, t=(0, 100, 0.01))

interactive(children=(FloatSlider(value=50.0, description='t', step=0.01), Output()), _dom_classes=('widget-in…

<function __main__.field(t)>

## Magnitude of the vector field (scalar field)

In [2]:
import numpy as np
from ipywidgets import interact
import matplotlib.pyplot as plt

c = 299792458
k = 1/(4 * np.pi * 8.85418782e-12)
q = 2 * 1.60217662e-19
r = 0.5
v = 20e6
frq = v/r

beta = v/c
accl = v**2/(r * c)

def R(x,y,pos):
    return np.sqrt(
        (x - r*np.cos(pos))**2 + (y - r*np.sin(pos))**2
    )

def u(x,y,t):
    _pos = frq * t
    
    pos = frq * (t - R(x,y,_pos)/c)
    
    RR = R(x,y,pos)
    
    nx = (x - r*np.cos(pos))/RR
    ny = (y - r*np.sin(pos))/RR
    
    vx = -beta * np.sin(pos)
    vy =  beta * np.cos(pos)
    
    ax = -accl * np.cos(pos)
    ay = -accl * np.sin(pos)
    
    num = k * q * ( (nx - vx)*ay - (ny - vy)*ax ) * ny
    den = c * ( 1 - nx*vx - ny*vy )**3 * RR
    
    return num/den

def v(x,y,t):
    _pos = frq * t
    
    pos = frq * (t - R(x,y,_pos)/c)
    
    RR = R(x,y,pos)
    
    nx = (x - r*np.cos(pos))/RR
    ny = (y - r*np.sin(pos))/RR
    
    vx = -beta * np.sin(pos)
    vy =  beta * np.cos(pos)
    
    ax = -accl * np.cos(pos)
    ay = -accl * np.sin(pos)
    
    num = k * q * ( (nx - vx)*ay - (ny - vy)*ax ) * nx
    den = c * ( 1 - nx*vx - ny*vy )**3 * RR
    
    return num/den
    
    
def field(t):
    t = t / 10000000

    n = 100
    img = np.zeros((n, n))
    zoom = 1

    for y in range(n):
        for x in range(n):
            X, Y = ((x-(n/2))/zoom, (y-(n/2))/zoom)
            if X**2 + Y**2 > (r*2)**2 or X**2 + Y**2 < (r-0.5)**2:
                img[y,x] = np.sqrt(u(X,-Y,t)**2 + v(X,-Y,t)**2)

    plt.imshow(img)
    pos = frq * t
    plt.scatter(
        r*np.cos(pos)*zoom+(n/2),
        -r*np.sin(pos)*zoom+(n/2)
    )
    plt.imshow(img)
    return plt.show()

interact(field, t=(0, 100, 0.01))

interactive(children=(FloatSlider(value=50.0, description='t', step=0.01), Output()), _dom_classes=('widget-in…

<function __main__.field(t)>